In [1]:
import pandas as pd
import numpy as np

import keras
from keras.preprocessing.text import Tokenizer
from keras.engine.topology import Layer
from keras import initializers as initializers, regularizers, constraints
from keras.callbacks import Callback
from keras.layers import Embedding, Input, Dense, LSTM, GRU, Bidirectional, TimeDistributed
from keras import backend as K
from keras.models import Model

from sklearn.metrics import roc_auc_score

import input as dataset
import tensorflow as tf

/Users/francescozanoli/anaconda2/envs/my-rdkit-env/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
-

In [3]:
MAX_WORD_LENGTH = 180
MAX_NB_CHARS = 180
EMBEDDING_DIM = 180
VALIDATION_SPLIT = 0.2

In [4]:
dataComp = dataset.LoadData('data',0)
smiles = list(map(lambda x: x._SMILE, dataComp))

Loading Started
Load of 24003 finished in 5.726010799407959s


In [5]:
tokenizer = Tokenizer(num_words=None, char_level=True)
tokenizer.fit_on_texts(smiles)
print(smiles[0])

O=C2N(Cc1ccccc1)CC(=O)C23(CC3)


In [6]:
data = np.zeros((len(smiles), MAX_WORD_LENGTH, MAX_WORD_LENGTH), dtype='int32')
for i, comp in enumerate(smiles):
    for j, char in enumerate(comp):
        try:
            if tokenizer.word_index[char] < MAX_NB_CHARS:
                data[i, j, k] = tokenizer.word_index[char]
        except:
               None
                    #print (char)

In [7]:
char_index = tokenizer.word_index
print('Total %s unique tokens.' % len(char_index))
print(char_index)

Total 53 unique tokens.
{'C': 1, 'c': 2, '(': 3, ')': 4, 'O': 5, '=': 6, '1': 7, 'N': 8, '2': 9, '3': 10, '[': 11, ']': 12, 'F': 13, '4': 14, 'l': 15, 'n': 16, 'S': 17, '@': 18, 'H': 19, '5': 20, '+': 21, '-': 22, 'B': 23, 'r': 24, '\\': 25, '#': 26, '6': 27, '.': 28, '/': 29, 's': 30, 'P': 31, '7': 32, 'i': 33, 'o': 34, '8': 35, 'I': 36, 'a': 37, '%': 38, '9': 39, '0': 40, 'K': 41, 'e': 42, 'A': 43, 'g': 44, 'p': 45, 'M': 46, 'T': 47, 'b': 48, 'd': 49, 'V': 50, 'Z': 51, 'G': 52, 'L': 53}


In [9]:
labels = keras.utils.to_categorical(list(map(lambda x: 1 if x.mutagen==True else 0,dataComp)),2)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (24003, 180, 180)
Shape of label tensor: (24003, 2)


In [10]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(0.2 * data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

print('Number of positive and negative reviews in traing and validation set')
print (y_train.sum(axis=0))
print (y_val.sum(axis=0))

Number of positive and negative reviews in traing and validation set
[12721.  6482.]
[3155. 1645.]


In [ ]:
embedding_layer = Embedding(len(char_index) + 1,
                            EMBEDDING_DIM,
                            input_length=MAX_WORD_LENGTH,
                            trainable=True)

char_input = Input(shape=(MAX_WORD_LENGTH,), dtype='int32')
char_sequences = embedding_layer(char_input)
char_lstm = Bidirectional(GRU(100, return_sequences=True))(char_sequences)
char_dense = TimeDistributed(Dense(200))(char_lstm)
char_att = AttentionWithContext()(char_dense)
charEncoder = Model(char_input, char_att)

words_input = Input(shape=(MAX_WORD_LENGTH, MAX_WORD_LENGTH), dtype='int32')
words_encoder = TimeDistributed(charEncoder)(words_input)
words_lstm = Bidirectional(GRU(100, return_sequences=True))(words_encoder)
words_dense = TimeDistributed(Dense(200))(words_lstm)
words_att = AttentionWithContext()(words_dense)
preds = Dense(2, activation='sigmoid')(words_att)
model = Model(words_input, preds)

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=1, batch_size=600)

Train on 19203 samples, validate on 4800 samples
Epoch 1/1
